# Steel Plate Defect Prediction

## Exploring datasets

In [1]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '.'

In [2]:
!kaggle competitions download -c playground-series-s4e3


  0%|          | 0.00/1.74M [00:00<?, ?B/s]
 57%|█████▋    | 1.00M/1.74M [00:00<00:00, 1.11MB/s]
100%|██████████| 1.74M/1.74M [00:01<00:00, 1.72MB/s]
100%|██████████| 1.74M/1.74M [00:01<00:00, 1.57MB/s]


In [3]:
from zipfile import ZipFile
with ZipFile('playground-series-s4e3.zip') as f:
    f.extractall('data')

In [4]:
import pandas as pd

In [5]:
raw_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')
sub_df = pd.read_csv('data/sample_submission.csv')

In [6]:
raw_df

,id,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,...,Orientation_Index,Luminosity_Index,SigmoidOfAreas,Pastry,Z_Scratch,K_Scatch,Stains,Dirtiness,Bumps,Other_Faults
0,0,584,590,909972,909977,16,8,5,2274,113,...,-0.5000,-0.0104,0.1417,0,0,0,1,0,0,0
1,1,808,816,728350,728372,433,20,54,44478,70,...,0.7419,-0.2997,0.9491,0,0,0,0,0,0,1
2,2,39,192,2212076,2212144,11388,705,420,1311391,29,...,-0.0105,-0.0944,1.0000,0,0,1,0,0,0,0
3,3,781,789,3353146,3353173,210,16,29,3202,114,...,0.6667,-0.0402,0.4025,0,0,1,0,0,0,0
4,4,1540,1560,618457,618502,521,72,67,48231,82,...,0.9158,-0.2455,0.9998,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19214,19214,749,757,143210,143219,17,4,4,2193,122,...,-0.1429,0.0044,0.2901,0,0,0,1,0,0,0
19215,19215,723,735,2488529,2488541,231,17,26,27135,104,...,0.7222,-0.0989,0.5378,0,0,0,0,0,0,1
19216,19216,6,31,1578055,1578129,780,114,98,71112,41,...,0.7719,-0.4283,0.9997,1,0,0,0,0,0,0
19217,19217,9,18,1713172,1713184,126,13,26,14808,88,...,0.9610,-0.1162,0.3509,0,0,0,0,0,0,1


In [7]:
raw_df['TypeOfSteel_A300'].value_counts()

TypeOfSteel_A300
0    11480
1     7739
Name: count, dtype: int64

In [8]:
raw_df['TypeOfSteel_A400'].value_counts()

TypeOfSteel_A400
1    11461
0     7758
Name: count, dtype: int64

In [9]:
raw_df.dtypes

id                         int64
X_Minimum                  int64
X_Maximum                  int64
Y_Minimum                  int64
Y_Maximum                  int64
Pixels_Areas               int64
X_Perimeter                int64
Y_Perimeter                int64
Sum_of_Luminosity          int64
Minimum_of_Luminosity      int64
Maximum_of_Luminosity      int64
Length_of_Conveyer         int64
TypeOfSteel_A300           int64
TypeOfSteel_A400           int64
Steel_Plate_Thickness      int64
Edges_Index              float64
Empty_Index              float64
Square_Index             float64
Outside_X_Index          float64
Edges_X_Index            float64
Edges_Y_Index            float64
Outside_Global_Index     float64
LogOfAreas               float64
Log_X_Index              float64
Log_Y_Index              float64
Orientation_Index        float64
Luminosity_Index         float64
SigmoidOfAreas           float64
Pastry                     int64
Z_Scratch                  int64
K_Scatch  

## Preprocessing

In [10]:
raw_df.drop(columns='id', inplace=True)

In [11]:
raw_df.describe()

,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,Maximum_of_Luminosity,...,Orientation_Index,Luminosity_Index,SigmoidOfAreas,Pastry,Z_Scratch,K_Scatch,Stains,Dirtiness,Bumps,Other_Faults
count,19219.000000,19219.000000,1.921900e+04,1.921900e+04,19219.000000,19219.000000,19219.000000,1.921900e+04,19219.000000,19219.000000,...,19219.000000,19219.000000,19219.000000,19219.000000,19219.000000,19219.000000,19219.000000,19219.000000,19219.000000,19219.000000
mean,709.854675,753.857641,1.849756e+06,1.846605e+06,1683.987616,95.654665,64.124096,1.918467e+05,84.808419,128.647380,...,0.102742,-0.138382,0.571902,0.076279,0.059837,0.178573,0.029554,0.025235,0.247828,0.341225
std,531.544189,499.836603,1.903554e+06,1.896295e+06,3730.319865,177.821382,101.054178,4.420247e+05,28.800344,14.196976,...,0.487681,0.120344,0.332219,0.265450,0.237190,0.383005,0.169358,0.156844,0.431762,0.474133
min,0.000000,4.000000,6.712000e+03,6.724000e+03,6.000000,2.000000,1.000000,2.500000e+02,0.000000,39.000000,...,-0.988400,-0.885000,0.119000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,49.000000,214.000000,6.574680e+05,6.575020e+05,89.000000,15.000000,14.000000,9.848000e+03,70.000000,124.000000,...,-0.272700,-0.192500,0.253200,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,777.000000,796.000000,1.398169e+06,1.398179e+06,168.000000,25.000000,23.000000,1.823800e+04,90.000000,127.000000,...,0.111100,-0.142600,0.472900,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1152.000000,1165.000000,2.368032e+06,2.362511e+06,653.000000,64.000000,61.000000,6.797800e+04,105.000000,135.000000,...,0.529400,-0.084000,0.999400,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,1705.000000,1713.000000,1.298766e+07,1.298769e+07,152655.000000,7553.000000,903.000000,1.159141e+07,196.000000,253.000000,...,0.991700,0.642100,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [18]:
input_cols = raw_df.columns.tolist()[:-7]
target_cols = raw_df.columns.tolist()[-7:]

In [19]:
target_cols

['Pastry',
 'Z_Scratch',
 'K_Scatch',
 'Stains',
 'Dirtiness',
 'Bumps',
 'Other_Faults']

In [23]:
from sklearn.preprocessing import MinMaxScaler

In [29]:
scaler = MinMaxScaler()
scaler.fit(raw_df[input_cols])

MinMaxScaler()

In [30]:
raw_df[input_cols] = scaler.transform(raw_df[input_cols])
test_df[input_cols] = scaler.transform(test_df[input_cols])

In [31]:
raw_df[input_cols].describe()

,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,Maximum_of_Luminosity,...,Outside_X_Index,Edges_X_Index,Edges_Y_Index,Outside_Global_Index,LogOfAreas,Log_X_Index,Log_Y_Index,Orientation_Index,Luminosity_Index,SigmoidOfAreas
count,19219.000000,19219.000000,19219.000000,19219.000000,19219.000000,19219.000000,19219.000000,19219.000000,19219.000000,19219.000000,...,19219.000000,19219.000000,19219.000000,19219.000000,19219.000000,19219.000000,19219.000000,19219.000000,19219.000000,19219.000000
mean,0.416337,0.438770,0.141981,0.141737,0.010992,0.012403,0.069982,0.016530,0.432696,0.418913,...,0.043866,0.609121,0.811902,0.591899,0.448949,0.375206,0.344566,0.551054,0.488912,0.514077
std,0.311756,0.292473,0.146642,0.146083,0.024437,0.023549,0.112033,0.038135,0.146941,0.066341,...,0.071281,0.225640,0.246889,0.482050,0.201418,0.173515,0.100550,0.246291,0.078806,0.377093
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.028739,0.122879,0.050132,0.050133,0.000544,0.001722,0.014412,0.000828,0.357143,0.397196,...,0.007685,0.443588,0.614749,0.000000,0.310161,0.259244,0.267572,0.361446,0.453474,0.152327
50%,0.455718,0.463429,0.107192,0.107192,0.001061,0.003046,0.024390,0.001552,0.459184,0.411215,...,0.012055,0.631088,0.960112,1.000000,0.383915,0.313430,0.327821,0.555275,0.486150,0.401703
75%,0.675660,0.679345,0.181907,0.181480,0.004238,0.008211,0.066519,0.005843,0.535714,0.448598,...,0.026522,0.782569,1.000000,1.000000,0.539366,0.419241,0.423375,0.766527,0.524524,0.999319
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [32]:
test_df[input_cols].describe()

,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,Maximum_of_Luminosity,...,Outside_X_Index,Edges_X_Index,Edges_Y_Index,Outside_Global_Index,LogOfAreas,Log_X_Index,Log_Y_Index,Orientation_Index,Luminosity_Index,SigmoidOfAreas
count,12814.000000,12814.000000,12814.000000,12814.000000,12814.000000,12814.000000,12814.000000,12814.000000,12814.000000,12814.000000,...,12814.000000,12814.000000,12814.000000,12814.000000,12814.000000,12814.000000,12814.000000,12814.000000,12814.000000,12814.000000
mean,0.416030,0.438429,0.141732,0.141384,0.010920,0.012510,0.070655,0.016602,0.431368,0.419107,...,0.043995,0.613078,0.814328,0.584985,0.448755,0.374843,0.344165,0.550257,0.488170,0.514644
std,0.311952,0.293004,0.147634,0.147248,0.024406,0.029126,0.208987,0.041522,0.146488,0.066734,...,0.073332,0.223581,0.246152,0.483308,0.201559,0.173431,0.099602,0.246517,0.077191,0.376815
min,0.000000,0.000000,0.000000,0.000000,-0.000013,0.000000,0.001109,0.000025,0.000000,-0.009346,...,0.000000,0.000000,-0.042793,0.000000,0.000000,0.016096,0.074629,-0.001313,-0.074586,0.000000
25%,0.026979,0.122879,0.048799,0.048774,0.000557,0.001722,0.014412,0.000838,0.346939,0.397196,...,0.007685,0.447113,0.627598,0.000000,0.312704,0.259244,0.267572,0.358871,0.453408,0.156640
50%,0.455132,0.463429,0.105707,0.105683,0.001068,0.002914,0.025499,0.001566,0.459184,0.411215,...,0.012055,0.633421,0.963911,1.000000,0.383915,0.313430,0.327821,0.555275,0.485299,0.401703
75%,0.675660,0.680515,0.181907,0.180997,0.004212,0.008211,0.065410,0.005755,0.535714,0.448598,...,0.026484,0.782569,1.000000,1.000000,0.539161,0.419241,0.423375,0.766527,0.524327,0.999092
max,0.990029,0.999415,1.000000,1.000000,1.000000,1.383525,20.123060,1.000000,1.035714,1.000000,...,1.123719,1.000000,1.000000,1.000000,1.033341,0.978192,1.055885,0.991011,0.966996,1.000000


In [33]:
inputs = raw_df[input_cols]
targets = raw_df[target_cols]

In [34]:
test_inputs = test_df[input_cols]

In [35]:
from sklearn.model_selection import train_test_split

In [36]:
train_inputs, val_inputs, train_targets, val_targets = train_test_split(inputs, targets,
                                                                        test_size=0.2,
                                                                        random_state=42)

## Training model

In [38]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier, XGBRFClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score

In [53]:
classifier = {
    'RF': RandomForestClassifier(),
    'GB': GradientBoostingClassifier(),
    'XGB': XGBClassifier(),
    'XGBRF': XGBRFClassifier(),
    'LGBM': LGBMClassifier()
}
new_classifier = classifier.copy()
result = {'Classifier': [], 'Accuracy': []}

In [54]:
for clf_name, clf in classifier.items():
    acc = 0
    for target_col in target_cols:
        clf = new_classifier.get(clf_name)
        clf.fit(train_inputs, train_targets[target_col])
        preds = clf.predict(val_inputs)
        acc += accuracy_score(val_targets[target_col], preds)
    acc = acc/len(target_cols)
    result['Classifier'].append(clf_name)
    result['Accuracy'].append(acc)
result_df = pd.DataFrame(result)
result_df

,Classifier,Accuracy
0,RF,0.892597
1,GB,0.894567
2,XGB,0.888843
3,XGBRF,0.890850
4,LGBM,0.894084


Ta chọn GB và LGBM để dự đoán

### Gradient Boosting

In [64]:
model = GradientBoostingClassifier()
predict = dict()
for target_col in target_cols:
    model.fit(train_inputs, train_targets[target_col])
    predict[target_col] = model.predict(val_inputs)
pred_df = pd.DataFrame(predict, columns=target_cols)
pred_df

,Pastry,Z_Scratch,K_Scatch,Stains,Dirtiness,Bumps,Other_Faults
0,0,0,0,0,0,0,1
1,0,0,1,0,0,0,0
2,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...
3839,0,1,0,0,0,0,0
3840,0,0,0,0,0,0,0
3841,0,0,0,0,0,0,0
3842,0,0,0,0,0,0,0


In [67]:
for target_col in target_cols:
    print(accuracy_score(val_targets[target_col], pred_df[target_col]))

0.9224765868886576
0.9539542143600416
0.9659209157127991
0.9820499479708636
0.9778876170655567
0.7819979188345474
0.6792403746097815


In [71]:
model = GradientBoostingClassifier()
for target_col in target_cols:
    model.fit(train_inputs, train_targets[target_col])
    preds = model.predict_proba(test_inputs)[:,1]
    sub_df[target_col] = preds

In [74]:
sub_df.to_csv('data/sub1.csv', index=None)

### LGBM

In [75]:
model = LGBMClassifier()
for target_col in target_cols:
    model.fit(train_inputs, train_targets[target_col])
    preds = model.predict_proba(test_inputs)[:,1]
    sub_df[target_col] = preds

In [77]:
sub_df.to_csv('data/sub2.csv', index=None)